In [ ]:
import numpy as np
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing, metrics
from importlib import reload
import sknn.mlp
import pickle
%matplotlib inline
pylab.rcParams['figure.figsize'] = (10, 6)
inline_rc = dict(mpl.rcParams)
import src.nnload as nnload
import src.nntrain as nntrain
import src.nnplot as nnplot


### Load data for training and cross-validation

In [ ]:
# Set script parameters
minlev = 0.25  # The top level of the atmosphere to consider.
rainonly = False
write_nn_to_netcdf = False # Option for writing neural network as a netcdf file

reload(nnload)
x, y, cv, Pout, lat, lev, dlev, timestep = nnload.loaddata('./data/convection_50day.pkl', 
                            minlev, all_lats=True, indlat=None, rainonly=False, noshallow=True)
# SHOW INPUT HISTOGRAM HERE TO GET A FEEL FOR DATAhy

### Preprocess Data

In [ ]:
# Show this explicitly
x_pp = nnload.init_pp(x_ppi, x)
x    = nnload.transform_data(x_ppi, x_pp, x)
y_pp = nnload.init_pp(y_ppi, y)
y    = nnload.transform_data(y_ppi, y_pp, y)
# Make preprocessor string for saving
pp_str =          'X-' + x_ppi['name'] + '-' + x_ppi['method'][:6] + '_'
pp_str = pp_str + 'Y-' + y_ppi['name'] + '-' + y_ppi['method'][:6] + '_'

### Build Neural Network

In [ ]:
# Show this explicitly
r_mlp, r_str = build_nn('regress', num_layers, 'Rectifier', hidneur,
                            'momentum', pp_str, batch_size=100,n_stable=n_stable,n_iter=n_iter,
                            learning_momentum=0.9,learning_rate=0.01,f_stable=.0007,
                            regularize=regularize, weight_decay=weight_decay)
print(r_str + ' Using ' + str(x.shape[0]) + ' training examples with ' + str(x.shape[1]) + \
          ' input features and ' + str(y.shape[1]) + ' output targets')
# Train neural network
r_mlp, r_errors = train_nn(r_mlp, r_str, x1, y1, w)
# Save neural network
pickle.dump([r_mlp, r_str, r_errors, x_ppi, y_ppi, x_pp, y_pp, lat, lev, dlev],
            open('./data/regressors/' + r_str + '.pkl', 'wb'))


### Plot figures

In [ ]:
# SHOW EXPLICITLY
# with validation data
nnplot.plot_all_figs(r_str)
nnplot.plot_all_figs(r_str, datasource='./data/convection_50day.pkl', validation=False)